In [215]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [216]:
import os

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12, extract_top_level_dict, MetaBatchNormLayer, MetaLinearLayer
from inner_loop_optimizers import LSLRGradientDescentLearningRule

In [217]:
# torch.ones(self.total_num_inner_loop_steps + 1) * self.init_learning_rate 확인
torch.ones(5 + 1) * 0.01

tensor([0.0100, 0.0100, 0.0100, 0.0100, 0.0100, 0.0100])

### 1. meta_neural_network_architectures.py의 extract_top_level_dict 이 무엇일까?

In [219]:
current_dict = {"layer_dict.step3.linear1.weights'" : 1, 
                'layer_dict.step3.linear1.bias': 2}

extract_top_level_dict(current_dict)

{'step3': {"linear1.weights'": 1, 'linear1.bias': 2}}

### 2. CNN 구현하기
#### (참조 : https://wikidocs.net/63565)

In [220]:
# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


In [221]:
# 합성곱층과 풀링 선언하기
conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) # 1채널 짜리를 입력받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) #32채널 짜리를 입력받아서 64채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1
print(conv2)

pool = nn.MaxPool2d(2)
pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
print(pool)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [222]:
# 구현체를 연결하여 모델 만들기
out = conv1(inputs)
print(out.shape) # 32채널의 28너비 28높이의 텐서, 32가 나온 이유는 conv1의 out_channel로 32를 지정, 패딩을 1폭으로 하고 3 × 3 커널을 사용하면 크기가 보존
out = pool(out)
print(out.shape)
out = conv2(out)
print(out.shape) # 64가 나온 이유는 conv2의 out_channel로 64를 지정
out = pool(out)
print(out.shape)

# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1) 
print(out.shape)

fc = nn.Linear(3136, 10) #input_dim = 3136, output_dim=10
out = fc(out)
print(out.shape)

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])
torch.Size([1, 3136])
torch.Size([1, 10])


In [223]:
class MyModule(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.build_network()
        
    def build_network(self):
        
        self.choices = nn.ModuleDict({
                'conv1': nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                'conv2': nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                'pool': nn.MaxPool2d(2)
        })
        self.activations = nn.ModuleDict([
                ['lrelu', nn.LeakyReLU()],
                ['prelu', nn.PReLU()]
        ])

    def forward(self, x, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x

In [224]:
cnn = MyModule()
cnn

MyModule(
  (choices): ModuleDict(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (activations): ModuleDict(
    (lrelu): LeakyReLU(negative_slope=0.01)
    (prelu): PReLU(num_parameters=1)
  )
)

### 3. ModuleDict 이해하기

In [225]:
class CNN_ModuleDict(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
        inputs = torch.Tensor(5, 3, 28, 28)
        self.input_shape = inputs.shape
        
        self.build_network()
        
    def build_network(self):
        
        x = torch.zeros(self.input_shape)
        out = x
        
        self.layer_dict = nn.ModuleDict()
        
        for i in range(5):
            print("out.size == ", out.size(1))
            self.layer_dict['conv{}'.format(i)] = nn.Conv2d(in_channels=out.size(1), out_channels=32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            
            out = self.layer_dict['conv{}'.format(i)](out)
        
        out = out.view(out.size(0), -1) 
        out = self.layer_dict['linear'] = nn.Linear(in_features= out.size(1), out_features = 10)
        
    def forward(self, x):
        print("forward")
        return x

In [226]:
inputs = torch.Tensor(1, 1, 28, 28)

model = CNN_ModuleDict()

model(inputs)

print(model)

out.size ==  3
out.size ==  32
out.size ==  32
out.size ==  32
out.size ==  32
forward
CNN_ModuleDict(
  (layer_dict): ModuleDict(
    (conv0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (linear): Linear(in_features=25088, out_features=10, bias=True)
  )
)


### 4. nn.Conv2d vs F.Conv2d 이해하기
#### 파이토치로 모델링을 할 때 nn을 사용할 때도 있고 nn.functional을 사용할 때도 있다.
#### nn.Conv2d에서 input_channel과 output_channel을 사용해서 연산했다면
#### functional.conv2d는 입력과 가중치 자체를 직접 넣어줌

In [227]:
import torch.nn.functional as F

inputs = torch.randn(64, 3, 244, 244)

conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
outputs1 = conv(inputs)
print("nn.Conv2d outputs1 === ", outputs1)

weight = conv.weight
bias = conv.bias
outputs2 = F.conv2d(inputs, weight, bias, padding=1)
print("F.Conv2d outputs2 === ", outputs2)

# F.Conv2d를 사용하게되면 filter (weight) 을 다양하게 사용할수있으며 만약에 nn.Conv2d랑 weight이 같다면 결과값도 같을 것 이다.

nn.Conv2d outputs1 ===  tensor([[[[-1.0671e-01,  2.6194e-01,  1.3226e-01,  ...,  5.4847e-01,
            3.3414e-01, -3.8444e-01],
          [ 1.1588e-01,  9.4492e-01,  4.2241e-01,  ...,  1.7307e+00,
            1.6741e-01, -3.6590e-01],
          [ 4.9526e-01, -5.0080e-01,  1.2607e+00,  ..., -4.8123e-02,
           -7.8493e-02, -1.8021e-01],
          ...,
          [ 1.7824e-01, -4.0100e-01, -3.3515e-01,  ...,  6.9650e-01,
           -1.0110e-01,  9.2544e-02],
          [-6.5434e-01,  3.7861e-01, -7.0563e-01,  ..., -1.5453e-01,
           -1.0219e+00,  3.5670e-01],
          [-2.5010e-02,  4.7007e-01, -4.2954e-01,  ..., -7.8428e-01,
           -7.5962e-01, -2.7130e-01]],

         [[-4.6079e-01, -4.8317e-01, -5.3503e-01,  ...,  5.2019e-01,
           -3.5852e-01, -1.3219e-01],
          [-5.4521e-01, -3.2071e-01, -7.9889e-01,  ...,  7.5632e-01,
           -7.6961e-01,  4.3210e-01],
          [-4.7794e-01,  6.0582e-01,  1.5520e-01,  ...,  5.3789e-01,
           -4.0715e-02,  4.0505e-0

F.Conv2d outputs2 ===  tensor([[[[-1.0671e-01,  2.6194e-01,  1.3226e-01,  ...,  5.4847e-01,
            3.3414e-01, -3.8444e-01],
          [ 1.1588e-01,  9.4492e-01,  4.2241e-01,  ...,  1.7307e+00,
            1.6741e-01, -3.6590e-01],
          [ 4.9526e-01, -5.0080e-01,  1.2607e+00,  ..., -4.8123e-02,
           -7.8493e-02, -1.8021e-01],
          ...,
          [ 1.7824e-01, -4.0100e-01, -3.3515e-01,  ...,  6.9650e-01,
           -1.0110e-01,  9.2544e-02],
          [-6.5434e-01,  3.7861e-01, -7.0563e-01,  ..., -1.5453e-01,
           -1.0219e+00,  3.5670e-01],
          [-2.5010e-02,  4.7007e-01, -4.2954e-01,  ..., -7.8428e-01,
           -7.5962e-01, -2.7130e-01]],

         [[-4.6079e-01, -4.8317e-01, -5.3503e-01,  ...,  5.2019e-01,
           -3.5852e-01, -1.3219e-01],
          [-5.4521e-01, -3.2071e-01, -7.9889e-01,  ...,  7.5632e-01,
           -7.6961e-01,  4.3210e-01],
          [-4.7794e-01,  6.0582e-01,  1.5520e-01,  ...,  5.3789e-01,
           -4.0715e-02,  4.0505e-01